In [5]:
import psycopg2
import pandas as pd

In [6]:
# Parámetros de conexión
host = "localhost"
port = "5432"
dbname = "EcommerceDB"
user = "postgres"
password = "postgres123"

# Crear conexión
try:
    conn = psycopg2.connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password
    )
    print(f"Conexión exitosa a la base de datos {dbname}")
except Exception as e:
    print(f"Error al conectar a la base de datos {dbname}:", e)


Conexión exitosa a la base de datos EcommerceDB


In [ ]:
# VARIACION DE PRECIOS PARA PRODUCTOS EN ORDENES
# Un mismo producto cuenta con varios precios registrados, en este analisis se tiene en cuenta el minimo y 
# maximo valor registrado para cada producto y se calcula el porcentaje de variacion

query = """
SELECT 
    d.productoid,
    p.precio AS precio_oficial,
    d.preciounitario
FROM 
    public.detalleordenes d
JOIN 
    public.productos p ON d.productoid = p.productoid
"""
df = pd.read_sql_query(query, conn)

df["diferencia_pct"] = ((df["preciounitario"] - df["precio_oficial"]) / df["precio_oficial"]) * 100

resumen = df.groupby("productoid")["diferencia_pct"].agg(["min", "max"]).reset_index()
resumen.rename(columns={"min": "min_diferencia_pct", "max": "max_diferencia_pct"}, inplace=True)

print(resumen.head())

   productoid  min_diferencia_pct  max_diferencia_pct
0           1          -97.114203           42.781222
1           2          -98.221527          -37.449312
2           3          -85.683514          283.898761
3           4          -28.352235         3209.939960
4           5          -67.691923         1150.062516


C:\Users\gabriel.b.alvarez\AppData\Local\Temp\ipykernel_9652\2238761405.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [ ]:
# EMAILS CON FORMATO INVALIDO
# En este analisis se encontró que hay emails donde se tienen espacios en blanco, lo cual los vuelve invalidos, 
# se calcula el porcentaje de estos con respecto al total

query_mal_formato = """ SELECT count(*)	FROM public.usuarios where email like '% %' """
query_totalidad = """ SELECT count(*) FROM public.usuarios """

cursor = conn.cursor()

cursor.execute(query_mal_formato)
resultado_mal_formato = cursor.fetchone()[0]
cursor.execute(query_totalidad)
resultado_totalidad = cursor.fetchone()[0]

porc_malos_sobre_total = (resultado_mal_formato/resultado_totalidad)*100

print(f'Porcentaje de emails con un formato incompleto: {porc_malos_sobre_total}%')

5.4%


In [ ]:
# ORDENES SIN DETALLE
# Al analizar los datos, se encontró con que hay ordenes que no tienen detalle, se procede a calcular el porcentaje
# que representan estos con respecto al total

query_cant_ord_sin_detalle = """ 
                    SELECT COUNT(*) FROM public.ordenes ord WHERE ord.ordenid NOT IN (SELECT DISTINCT ordenid from public.detalleordenes) 
                    """
query_cant_ord = """ SELECT COUNT(distinct ordenid) FROM public.ordenes ord """

cursor = conn.cursor()

cursor.execute(query_cant_ord_sin_detalle)
resultado_cant_ord_sin_detalle = cursor.fetchone()[0]

cursor.execute(query_cant_ord)
resultado_cant_ord = cursor.fetchone()[0]

porc_ord_sin_detalle = (resultado_cant_ord_sin_detalle/resultado_cant_ord)*100

print(f'Porcentaje de ordenes sin detalle con respecto al total: {porc_ord_sin_detalle}%')

90.0%


In [ ]:
# DIFERENCIAS DE TOTALES DE ORDENES
# Durante el analisis de los datos, se notó que las ordenes no coinciden en ambas tablas con respecto al total
# registrado, se procede a comprobarlo y demostrar la diferencia entre estos

query = """
SELECT 
	ord.ordenid,
	ord.total as total_orden,
	det_ord.total_orden as total_orden_detalle
FROM 
	public.ordenes ord
INNER JOIN
	(SELECT ordenid, SUM(cantidad * preciounitario) as total_orden
	FROM public.detalleordenes
	GROUP BY ordenid) det_ord ON ord.ordenid = det_ord.ordenid
"""
df = pd.read_sql_query(query, conn)

df['diferencia_total'] = (df['total_orden'] - df['total_orden_detalle']).abs()
df = df.sort_values(by='diferencia_total', ascending=False)

print(df)

     ordenid  total_orden  total_orden_detalle  diferencia_total
975      976       456.57             19226.99          18770.42
428      429       970.02             18546.36          17576.34
371      372       317.60             17722.88          17405.28
697      698       381.63             17414.44          17032.81
711      712        42.47             17057.27          17014.80
..       ...          ...                  ...               ...
388      389       999.85              1904.60            904.75
614      615       830.68              1622.83            792.15
958      959       424.84              1088.10            663.26
329      330       733.68              1172.63            438.95
106      107       877.89              1107.47            229.58

[1000 rows x 4 columns]


C:\Users\gabriel.b.alvarez\AppData\Local\Temp\ipykernel_21048\3093765353.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [ ]:
# CIUDADES EN MULTIPLES DEPARTAMENTOS Y PROVINCIAS
# En el analisis de los datos, se encontró con que varias ciudades se encuentran en simultaneo en distintos
# departamentos, lo cual esta mal. Se comprueba demostrando la cantidad de departamentos por ciudad

query = """
select 
    ciudad, count(distinct departamento) as cant_departamentos
from 
    public.direccionesenvio
group by 
    ciudad
"""
df = pd.read_sql_query(query, conn)
df = df.sort_values(by='cant_departamentos', ascending=False)

print(df)

                                ciudad  cant_departamentos
0                         Bahía Blanca                  21
23               San Miguel de Tucumán                  21
20  San Ferando del Valle de Catamarca                  20
5                           Corrientes                  20
2                   Comodoro Rivadavia                  19
16                         Resistencia                  19
15                              Rawson                  19
26                          Santa Rosa                  19
28                             Ushuaia                  19
17                        Río Gallegos                  19
14                             Posadas                  19
4                              Córdoba                  18
7                             La Plata                  18
6                              Formosa                  18
27                 Santiago del Estero                  18
25                            Santa Fe                  

C:\Users\gabriel.b.alvarez\AppData\Local\Temp\ipykernel_21048\1538330939.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


In [14]:
conn.close()